# Creation d'un premier reseau à trois couches et 3 paramètres
Résolution du problème inverse d'intégrale multiple
Pour un réseau de neurones Unfolding
Chaque couche n'a que trois paramètres.

On rappelle le modèle.
We set an initial value $x_0$ and
we introduce the following $m$-layer neural network

\begin{equation}
	\label{def:modelG}
	\begin{cases}
	\textbf{Initialization:} \\
	\quad b_0 = T^*y^\delta ,\\
	\textbf{Layer $n\in \{1,\ldots,m\}$:} \\
      \quad x_n %=  Q_n(x_0,x_{n-1}) 
          = R_n(W_n x_{n-1} + W_{n,0}b_0)\;,
    \end{cases}		  
\end{equation}
where 
\begin{align}
&R_n = \text{prox}_{\lambda_n \mu_n g}\\
&W_n =  1 - \lambda_n T^*T - \lambda_n \alpha_n D^*D \\
&W_{n,0} = \lambda_n 1.
\end{align}

Dansd un premier temps on apprend seulement le paramètre $\alpha$

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random as rand
%matplotlib inline

## Data
1 - first create the panda frame (or numpy)

2 - transform into tensor

In [2]:
# Physical data
l = 200
tau = 100
dep = 2
# Numerical data
nx = 200
dx = l/(nx+1)
nt = 200
dt = tau/nt
T_operator = 1/100*dx*np.tri(nt, nx, 0, dtype=int)
# Data sample
nsamp = 150
x_dagger = np.zeros((nx,nsamp))
y = np.zeros((nt,nsamp))
x_grid = np.linspace(0,l,nx)
#
x_sample = np.zeros((nx,nsamp))
#
for i in range(0,nsamp):
    mu = l/2
    sigma = 0.1
    x_dagger[:,i] = (sigma*np.sqrt(2*np.pi))**-1*np.exp(-(x_grid-mu)**2/2*sigma**2)
    y[:,i] = T_operator.dot(x_dagger[:,i]) 
    xi = np.random.uniform(-0.05,0.05,nt)
    y[:,i] += 0.05*xi*np.linalg.norm(y[:,i])/np.linalg.norm(xi)
    x_sample[:,i] = np.transpose(T_operator).dot(y[:,i])

In [3]:
# Train-Test split
train_X = x_sample[:,:100]
test_X = x_sample[:,100:]
train_y = x_dagger[:,:100]
test_y = x_dagger[:,100:]
# Convert to pytorch tensor
X_train = torch.from_numpy(train_X)
X_test = torch.from_numpy(test_X)
y_train = torch.from_numpy(train_y)
y_test = torch.from_numpy(test_y)

In [4]:
print(X_test)

tensor([[0.9865, 0.9872, 0.9954,  ..., 0.9982, 0.9896, 0.9951],
        [0.9861, 0.9871, 0.9959,  ..., 0.9978, 0.9896, 0.9952],
        [0.9863, 0.9869, 0.9954,  ..., 0.9982, 0.9899, 0.9952],
        ...,
        [0.0296, 0.0288, 0.0310,  ..., 0.0290, 0.0288, 0.0298],
        [0.0197, 0.0188, 0.0206,  ..., 0.0198, 0.0194, 0.0199],
        [0.0093, 0.0096, 0.0102,  ..., 0.0096, 0.0098, 0.0097]],
       dtype=torch.float64)


In [5]:
# Regularisation operator
D_operator = np.diag(np.ones(nx-1),1)+ np.diag(np.ones(nx-1),-1)-2*np.eye(nx)

## Network Model

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        
    def forward(self, x, T, D, nx):
        tensor_grad = torch.eye(nx)\
        - torch.from_numpy(np.transpose(T).dot(T)) \
        - self.alpha*torch.from_numpy(np.transpose(D).dot(D))
        y_pred = torch.matmul(tensor_grad,x)
        return y_pred

In [7]:
model = Model()

In [8]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([-1.3007], requires_grad=True)


In [9]:
print(X_train[:,1].size())
y_pred = model.forward(X_train[:,1],T_operator,D_operator,nx)
y_pred.size()
y_pred.norm().backward()
model.alpha.grad

torch.Size([200])


tensor([-0.7484])

## Training

1 - Set the loss function

2 - Set the optimization


In [10]:
criterion = nn.MSELoss(reduction='mean')
criterion(y_train[:,99], y_pred)

tensor(2.0518, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [11]:
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad,model.parameters()), lr = 0.001)

In [12]:
epochs = 50
losses = []

for i in range(epochs):
    i+=1
    k = rand.randint(0,99)
    y_pred = model.forward(X_train[:,k],T_operator,D_operator,nx)
    loss = criterion(y_pred, y_train[:,1])
    losses.append(loss)
    #
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f'epoch: {i:2} loss: {loss.item():10.8f} weight: {model.alpha.item() :10.8f}')

epoch:  1 loss: 2.05564942 weight: -1.29985523
epoch:  2 loss: 2.05530897 weight: -1.29979897
epoch:  3 loss: 2.05223881 weight: -1.29974294
epoch:  4 loss: 2.05208899 weight: -1.29968667
epoch:  5 loss: 2.04390520 weight: -1.29963195
epoch:  6 loss: 2.05387928 weight: -1.29957569
epoch:  7 loss: 2.05328127 weight: -1.29951954
epoch:  8 loss: 2.04785301 weight: -1.29946375
epoch:  9 loss: 2.06287786 weight: -1.29940629
epoch: 10 loss: 2.04563425 weight: -1.29935098
epoch: 11 loss: 2.05429536 weight: -1.29929471
epoch: 12 loss: 2.04740326 weight: -1.29923987
epoch: 13 loss: 2.05178695 weight: -1.29918444
epoch: 14 loss: 2.05165940 weight: -1.29912806
epoch: 15 loss: 2.05497582 weight: -1.29907191
epoch: 16 loss: 2.04806677 weight: -1.29901636
epoch: 17 loss: 2.06311915 weight: -1.29895937
epoch: 18 loss: 2.04386546 weight: -1.29890478
epoch: 19 loss: 2.05435282 weight: -1.29884863
epoch: 20 loss: 2.05496013 weight: -1.29879248
epoch: 21 loss: 2.04203310 weight: -1.29873788
epoch: 22 los

## Two Layers feedforward network, try #1

In [13]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

99 2.4724090099334717
199 0.05619945377111435
299 0.0021141788456588984
399 9.78464086074382e-05
499 5.4044739954406396e-06
